In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

import tiktoken

from transformers import Trainer, TrainingArguments
from datasets import Dataset

from transformers import AutoModelForCausalLM, AutoTokenizer


In [ ]:

tokenizer = AutoTokenizer.from_pretrained("/output/path")
model = AutoModelForCausalLM.from_pretrained("/output/path")

# Sample QA data
qa_data = [
    {"question": "What is the capital of France?", "answer": "Paris"},
    {"question": "Who wrote 'Hamlet'?", "answer": "William Shakespeare"},
    # Add more question-answer pairs...
]

# Convert to the format expected for fine-tuning
def format_qa(example):
    return {
        "input_text": f"Question: {example['question']} Answer:",
        "target_text": f" {example['answer']}"  # Add a space before the answer
    }

formatted_qa_data = [format_qa(qa) for qa in qa_data]

# Load LLaMA model and tokenizer (replace with your model path)
tokenizer = LLaMATokenizer.from_pretrained("path/to/llama-model")
model = LLaMAForCausalLM.from_pretrained("path/to/llama-model")

# Prepare dataset and tokenize
def tokenize_function(examples):
    inputs = tokenizer(examples['input_text'], padding="max_length", truncation=True, max_length=128)
    targets = tokenizer(examples['target_text'], padding="max_length", truncation=True, max_length=128)
    inputs['labels'] = targets['input_ids']
    return inputs

# Create Hugging Face dataset
dataset = Dataset.from_list(formatted_qa_data)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir="./llama-finetuned-qa",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_total_limit=2,
    logging_dir="./logs",
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()
